In [1]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_rows = 5000

In [3]:
import os
os.listdir()

['dataset', '.ipynb_checkpoints', '.git', 'beneran.ipynb', 'Untitled.ipynb']

In [4]:
train_df = pd.read_csv('dataset/train.csv', index_col='ID')

In [22]:
test_df = pd.read_csv('dataset/test.csv', index_col='ID')

In [6]:
train_df.head()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4459 entries, 000d6aaf2 to ffeb15d25
Columns: 4992 entries, target to 9fc776466
dtypes: float64(1845), int64(3147)
memory usage: 169.9+ MB


In [8]:
train_feat = train_df.loc[:, train_df.columns != 'target']
target = train_df['target']

In [9]:
dropped_cols = []
for col in train_feat.columns:
    std = train_feat[col].describe().loc['std']
    if std == 0:
        dropped_cols.append(col)

In [10]:
train_feat.drop(labels=dropped_cols, axis=1, inplace=True)

/home/teguh/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [11]:
train_feat.shape

(4459, 4735)

In [12]:
duplicate_cols = []
for col in train_feat.columns:
    for comp_col in train_feat.columns:
        if (col != comp_col) and np.array_equal(train_feat[col].values, train_feat[comp_col].values):
            print('found duplicate of {}: {}'.format(col, comp_col))
            duplicate_cols.append(comp_col)

found duplicate of 34ceb0081: d60ddde1b
found duplicate of 8d57e2749: acc5b709d
found duplicate of 8d57e2749: f333a5f60
found duplicate of 168b3e5bc: f8d75792f
found duplicate of d60ddde1b: 34ceb0081
found duplicate of a765da8bc: 912836770
found duplicate of 912836770: a765da8bc
found duplicate of acc5b709d: 8d57e2749
found duplicate of acc5b709d: f333a5f60
found duplicate of f8d75792f: 168b3e5bc
found duplicate of f333a5f60: 8d57e2749
found duplicate of f333a5f60: acc5b709d


In [13]:
train_feat.drop(labels=duplicate_cols, axis=1, inplace=True)

/home/teguh/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
train_feat = train_feat.T.drop_duplicates().T

In [15]:
train_feat.head()

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000fbd867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2200000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0028cbf45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
len(train_feat)

4459

In [17]:
non_zero_perc = []
low_non_zero = []
for col in train_feat.columns:
    perc = train_feat[col].astype(bool).sum(axis=0)/4459.0*100
    non_zero_perc.append(perc)
#     print('column: {}  non zero values: {:.2f}%'.format(col, perc))
    if perc <= 1.0:
        low_non_zero.append(col)

In [18]:
non_zero_perc = np.array(non_zero_perc)

In [27]:
len(non_zero_perc[non_zero_perc <= 1.0])

2099

In [20]:
train_feat.drop(labels=low_non_zero, axis=1, inplace=True)

In [21]:
target = np.log1p(target)

In [24]:
test_df = test_df[train_feat.columns]

In [25]:
test_df.shape

(49342, 2627)

In [26]:
train_feat.shape

(4459, 2627)

In [28]:
def add_statistics(data, test):
    # This is part of the trick I think, plus lightgbm has a special process for NaNs
#     data.replace(0, np.nan, inplace=True)
#     test.replace(0, np.nan, inplace=True)
    
#     original_features = [f for f in data.columns if f not in ['target', 'ID']]
    for df in [data, test]:
#         df['nb_nans'] = df[original_features].isnull().sum(axis=1)
        # All of the stats will be computed without the 0s 
        df['the_median'] = df[original_features].median(axis=1)
        df['the_mean'] = df[original_features].mean(axis=1)
        df['the_sum'] = df[original_features].sum(axis=1)
        df['the_std'] = df[original_features].std(axis=1)
        df['the_kur'] = df[original_features].kurtosis(axis=1)
        
    return data, test

In [29]:
train_feat, test_df = add_statistics(train_feat, test_df)

/home/teguh/anaconda3/lib/python3.6/site-packages/pandas/core/nanops.py:622: RuntimeWarning: divide by zero encountered in true_divide
  mean = values.sum(axis, dtype=np.float64) / count


In [35]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import make_scorer

In [33]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.power(y_true - pred, 2)))

In [34]:
X_train, X_test, y_train, y_test = train_test_split(train_feat, target, test_size=0.2, shuffle=True)

In [40]:
def rmsle_cv(model):
    kf = KFold(5, shuffle=True, random_state=42).get_n_splits(train_feat.values)
    rmse = np.sqrt(-cross_val_score(model, train_feat, target, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [42]:
model = XGBRegressor(colsample_bytree=0.055, colsample_bylevel =0.5,
                    gamma=1.5, learning_rate=0.02, max_depth=32, 
                    objective='reg:linear',booster='gbtree',
                    min_child_weight=57, n_estimators=1000, reg_alpha=0, 
                    reg_lambda = 0,eval_metric = 'rmse', subsample=0.7, 
                    silent=1, n_jobs = -1, early_stopping_rounds = 14,
                    random_state =7, nthread = -1)

In [43]:
scores = rmsle_cv(model)

array([1.33396843, 1.41902816, 1.29294772, 1.35960952, 1.43032931])

In [44]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.055, early_stopping_rounds=14,
       eval_metric='rmse', gamma=1.5, learning_rate=0.02, max_delta_step=0,
       max_depth=32, min_child_weight=57, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=-1, objective='reg:linear', random_state=7,
       reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.7)

In [49]:
non_zero_importance_cols = pd.DataFrame({'feature': X_train.columns, 'importance': model.feature_importances_}).sort_values(
    by=['importance'], ascending=[False])

In [51]:
non_zero_importance_cols = non_zero_importance_cols[non_zero_importance_cols.importance > 0]

In [54]:
train_feat = train_feat[non_zero_importance_cols.feature.values]
test_df = test_df[non_zero_importance_cols.feature.values]

In [55]:
test_df.head()

,the_mean,the_std,the_kur,the_sum,the_median,nb_nans,eeb9cd3aa,fb0f5dbfe,fb49e4212,58e2e02e6,...,1bf6240eb,2bbcbf526,82775fc92,1a82869a6,0fe78acfa,b02dfb243,f9378f7ef,31015eaab,75d240f7b,c3c3f66ff
ID,,,,,,,,,,,,,,,,,,,,,
000137c73,4.455847e+07,1.490285e+08,36.016985,1.916014e+09,6.877071e+06,2584,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00021489f,1.068873e+07,1.825299e+07,NaN,3.206618e+07,2.986793e+05,2624,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0004d7953,5.372810e+06,1.053854e+07,29.779054,4.620616e+08,2.000000e+06,2541,NaN,NaN,NaN,NaN,...,2000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00056a333,1.556117e+07,1.401439e+07,10.691789,1.758413e+09,1.140800e+07,2514,40000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00056d8eb,4.896387e+06,4.002743e+05,NaN,9.792775e+06,4.896387e+06,2625,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
model = XGBRegressor(colsample_bytree=0.055, colsample_bylevel =0.5,
                    gamma=1.5, learning_rate=0.02, max_depth=32, 
                    objective='reg:linear',booster='gbtree',
                    min_child_weight=57, n_estimators=1000, reg_alpha=0, 
                    reg_lambda = 0,eval_metric = 'rmse', subsample=0.7, 
                    silent=1, n_jobs = -1, early_stopping_rounds = 14,
                    random_state =7, nthread = -1)

In [58]:
scores = rmsle_cv(model)

In [60]:
model = XGBRegressor(colsample_bytree=0.055, colsample_bylevel =0.5,
                    gamma=1.5, learning_rate=0.02, max_depth=32, 
                    objective='reg:linear',booster='gbtree',
                    min_child_weight=57, n_estimators=1000, reg_alpha=0, 
                    reg_lambda = 0,eval_metric = 'rmse', subsample=0.7, 
                    silent=1, n_jobs = -1, early_stopping_rounds = 14,
                    random_state =7, nthread = -1)
model.fit(train_feat, target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.055, early_stopping_rounds=14,
       eval_metric='rmse', gamma=1.5, learning_rate=0.02, max_delta_step=0,
       max_depth=32, min_child_weight=57, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=-1, objective='reg:linear', random_state=7,
       reg_alpha=0, reg_lambda=0, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.7)

In [61]:
y_pred = model.predict(test_df)

In [62]:
y_pred = np.expm1(y_pred)

In [63]:
y_pred[:10]

array([2457946.2, 3091927. , 2123717. , 5849769.5, 2373918. , 2872090.5,
       2762381.2, 2477096.5, 4642790. , 3871021.5], dtype=float32)

In [ ]:
sub = pd.DataFrame()
sub['ID'] = test_df['ID']
sub['target'] = ensemble
sub.to_csv('submission.csv',index=False)